In [1]:
!nvidia-smi

Tue Nov 24 13:19:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.chdir('/content/drive/My Drive/GitHub/mitochondria_segmentation/src')
print(os.getcwd())
#print(os.listdir())

/content/drive/My Drive/GitHub/mitochondria_segmentation/src


In [ ]:
!python train.py

Data loaded.
device: cuda:0
train.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32)
train.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lables = torch.tensor(lables, dtype=torch.long)
[431, loss: 0.003282]
6.0 minutes per epoch.
[432, loss: 0.003289]
2.0 minutes per epoch.
[433, loss: 0.003328]
2.0 minutes per epoch.
[434, loss: 0.003318]
2.0 minutes per epoch.
[435, loss: 0.003283]
2.0 minutes per epoch.
[436, loss: 0.003297]
2.0 minutes per epoch.
[437, loss: 0.003299]
2.0 minutes per epoch.
[438, loss: 0.003339]
3.0 minutes per epoch.
[439, loss: 0.003223]
2.0 minutes per epoch.
[440, loss: 0.003249]
2.0 minutes per epoch.
[

In [3]:
!python predict.py

In [ ]:
!python messure.py

{'TP': 2595339, 'TN': 92643630, 'FP': 177339, 'FN': 1314828, 'acc': 0.9845740775751873, 'sn': 0.9360405355400083, 'sp': 0.9860062837557424, 'rc': 0.663741216168005, 'f1': 0.7767168020140198, 'jac': 0.6349443890724563, 'dice': 0.7767168024995342}
